# Instructions

The places where you have enter code, to answer the questions, are marked with `# YOUR CODE HERE`. Once you have written your code you should remove any `raise NotImplementedError()` statements.


## Question 1 (4 points)

Suppose that we would like to minimize $f(x_1,x_2,x_3)=-5x_1-2x_1x_2 -3x_3$ which is subjected to the constraint $x_1+ x_2 + x_3 = 1$. 

Complete the function `constrained_bqm` so that it returns the corresponding BQM. Lagrange multiplier should be provided as a parameter.

Note: Name your variables in the format `x_i` and use BQM functionalities to incorporate the constraint.

The function `constrained_bqm` has

- Input:
    - `L`: a float representing the Lagrange multiplier
- Returns:
    - A BQM instance


In [16]:
%pip install dimod


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
from dimod import BQM

def constrained_bqm(L):
    # YOUR CODE HERE

    # Define the variables
    variables = ['x_1', 'x_2', 'x_3']
    
    # Define the objective function coefficients
    obj_coeffs = {'x_1': -5, 'x_2': -2, 'x_3': -3}
    
    # Initialize the BQM
    bqm = BQM('BINARY')
    
    # Add objective function terms
    for var, coeff in obj_coeffs.items():
        bqm.add_variable(var, coeff)
    
    # Add constraint penalty term
    for var in variables:
        bqm.add_linear_equality_constraint({var: 1}, constant=1, lagrange_multiplier=L)
    

    # raise NotImplementedError()
    # Do not modify anything below this line
    return bqm


In [20]:
# You can use this cell to call and check the output of the function

print(constrained_bqm(2))


ValueError: too many values to unpack (expected 2)

In [ ]:
# hidden tests will be used for grading.

In [ ]:
# hidden tests will be used for grading.

## Question 2 (4 points)

Next questions are about the Knapsack Problem.

Given $n$ items, each with an associated weight $w_i$ and cost $c_i$, the goal of the Knapsack problem is to pack as many items to maximize the value of the packed knapsack, while not exceeding the knapsack capacity.

Let $x_i$ be a binary variable that is equal to 1 if $i$'th item is selected and 0 otherwise. 

In Knapsack problem, the objective is to maximize
$$
\sum c_ix_i
$$
subject to the constraint
$$
\sum w_ix_i \leq W. 
$$

However, note that when formulating a QUBO, we always consider the equivalent `minimization` problem. 

Complete the function `knapsack_objective` which takes as parameter a BQM instance and the list of costs and adds the objective function to be *minimized* to the input BQM.

Note: Name your variables in the format `x_i` and use BQM functionalities.

The function `knapsack_objective` has

- Inputs:
    - `bqm`: a BQM instance,
    - `costs`: a list of $n$ elements corresponding to costs of the items.
            
- Returns:
    - Modified `bqm`


In [ ]:
from dimod import BQM

def knapsack_objective(bqm, costs):
    # YOUR CODE HERE

    # Add the objective function to the BQM
    for i, cost in enumerate(costs):
        bqm.add_variable(f'x_{i}', cost)

    # raise NotImplementedError()
    # Do not modify anything below this line
    return bqm


In [ ]:
# You can use this cell to call and check the output of the function
from dimod import BQM

bqm = BQM("BINARY")
costs = [2, 7, 3, 4, 6]
bqm = knapsack_objective(bqm, costs)
print(bqm)


In [ ]:
# hidden tests will be used for grading.

In [ ]:
# hidden tests will be used for grading.

## Question 3 (4 points)

Complete the function `knapsack_constraint` which takes as parameter a BQM instance and the list of costs, weights and capacity of the knapsack and adds the constraint to the input BQM.

Note: Name your variables in the format `x_i` and use BQM functionalities. Set Lagrange multiplier $L$ such that $L = \max\{c_i\} + 1$. Set `label="c"`.

The function `knapsack_constraint` has

- Inputs:
    - `bqm`: a BQM instance,
    - `weights`: a list of $n$ elements corresponding to the weights of the items,
    - `costs`-a list of $n$ elements corresponding to costs of the items,
    - `W`-capacity of Knapsack
            
- Returns:
    - Modified `bqm`


In [ ]:
from dimod import BQM

def knapsack_constraint(bqm, weights, costs, W):
    # YOUR CODE HERE

    # Calculate the Lagrange multiplier
    L = max(costs) + 1

    # Add the constraint term to the BQM
    bqm.add_linear_equality_constraint(
        [(f'x_{i}', weights[i]) for i in range(len(weights))],
        constant=-W,
        lagrange_multiplier=L
    )

    # raise NotImplementedError()
    # Do not modify anything below this line
    return bqm


In [ ]:
# You can use this cell to call and check the output of the function
from dimod import BQM

bqm = BQM("BINARY")
costs = [2, 7, 3, 4, 6]
W = 10
weights = [1, 4, 8, 2, 3]
bqm = knapsack_constraint(bqm, weights, costs, W)
print(bqm)

In [ ]:
# hidden tests will be used for grading.

In [ ]:
# hidden tests will be used for grading.

## Question 4 (4 points)

Complete the function `knapsack_bqm` which takes as parameter the list of costs, weights and capacity of the knapsack and returns a BQM instance for the given knapsack problem.

Note: Name your variables in the format `x_i` and use the functions you have written previously.

The function `knapsack_bqm` has

- Inputs:
    - `weights`: a list of $n$ elements corresponding to the weights of the items,
    - `costs`-a list of $n$ elements corresponding to costs of the items,
    - `W`: capacity of Knapsack.
            
- Returns:
    - A BQM instance `bqm`.


In [ ]:
from dimod import BQM

def knapsack_bqm(weights, costs, W):
    # YOUR CODE HERE

    # Create a BQM instance
    bqm = BQM('BINARY')

    # Add the objective function
    bqm = knapsack_objective(bqm, costs)

    # Add the constraint
    bqm = knapsack_constraint(bqm, weights, costs, W)

    # raise NotImplementedError()
    # Do not modify anything below this line
    return bqm


In [ ]:
# You can use this cell to call and check the output of the function
from dimod import BQM
bqm = BQM("BINARY")
costs = [2, 7, 3, 4, 6]
W = 10
weights = [1, 4, 8, 2, 3]
bqm = knapsack_bqm(weights, costs, W)
print(bqm)


In [ ]:
# hidden tests will be used for grading.

In [ ]:
# hidden tests will be used for grading.

## Question 5 (4 points)

Complete the function `anneal` which takes as parameter a BQM instance and number of reads and returns a sampleset object after running simulated annealing. Make the necessary imports.

The function `anneal` has

- Inputs:
    - `bqm`: a BQM instance,
    - `num_reads`: an integer coresponding to the number of reads for simulated annealing experiment.
            
- Returns:
    - A `SampleSet` instance 


In [ ]:
from dimod import SimulatedAnnealingSampler

def anneal(bqm, num_reads):
    # YOUR CODE HERE

    # Initialize the simulated annealing sampler
    sampler = SimulatedAnnealingSampler()

    # Sample from the BQM using simulated annealing
    sampleset = sampler.sample(bqm, num_reads=num_reads)

    # raise NotImplementedError()
    # Do not modify anything below this line
    return sampleset


In [ ]:
# You can use this cell to call and check the output of the function

linear = {'x1': 5, 'x2': -3}
quadratic = {('x1', 'x2'): 7}
offset = 2
vartype = 'BINARY'

bqm = BQM(linear, quadratic, offset, vartype)
sampleset = anneal(bqm, 1000)
print(sampleset)


In [ ]:
# hidden tests will be used for grading.

In [ ]:
# hidden tests will be used for grading.

## Question 6 (4 points)

Complete the function `knapsack_solution` which takes as parameter number of reads, the list of costs, weights and weight of the knapsack and returns the sample with the lowest energy for the given Knapsack instance.

Note: You should use the functions `anneal` and `knapsack_bqm` you have written previously.

The function `knapsack_solution` has

- Inputs:
    - `num_reads`: number of reads,
    - `weights`: a list of $n$ elements corresponding to the weights of the items,
    - `costs`-a list of $n$ elements corresponding to costs of the items,
    - `W`-capacity of Knapsack

- Returns:
    - `solution`: a tuple consisting of solution vector and energy


In [ ]:
def knapsack_solution(num_reads,weights, costs, W):
    # YOUR CODE HERE

    # Create a BQM instance for the knapsack problem
    bqm = knapsack_bqm(weights, costs, W)
    
    # Use simulated annealing to find the sample with the lowest energy
    sampleset = anneal(bqm, num_reads)
    
    # Extract the solution vector and its corresponding energy from the sample set
    solution = sampleset.first

    # raise NotImplementedError()
    # Do not modify anything below this line
    return solution


In [ ]:
# You can use this cell to call and check the output of the function

costs = [2, 7, 3, 4, 6]
W = 10
weights = [1, 4, 8, 2, 3]
num_reads = 100
solution = knapsack_solution(num_reads,weights, costs, W)
print(solution)


In [ ]:
# hidden tests will be used for grading.